In [21]:
# 1. Data Loading

import pandas as pd
df = pd.read_csv('train.csv')
print("data loaded successfully")
print("shape:", df.shape)
print("info:")
df.info()
df.head(5)


data loaded successfully
shape: (891, 12)
info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# 2. Column summary

column_summary = pd.DataFrame({
    'column': df.columns,
    'Dtype': df.dtypes,
    'Missing': df.isnull().sum(),
    'Unique': df.nunique()
})
summary = column_summary.sort_values(by='Missing', ascending=False)
print(summary)

                  column    Dtype  Missing  Unique
Cabin              Cabin   object      687     147
Age                  Age  float64      177      88
Embarked        Embarked   object        2       3
PassengerId  PassengerId    int64        0     891
Name                Name   object        0     891
Pclass            Pclass    int64        0       3
Survived        Survived    int64        0       2
Sex                  Sex   object        0       2
Parch              Parch    int64        0       7
SibSp              SibSp    int64        0       7
Fare                Fare  float64        0     248
Ticket            Ticket   object        0     681


In [30]:
# 3. Value counts & proportions

columns = ['Pclass', 'Sex', 'Embarked']
summary = pd.DataFrame()
for col in columns:
    counts = df[col].value_counts()
    percentages = (counts / len(df) * 100)
    temp_df = pd.DataFrame({
        f'{col}_Count' : counts,
        f'{col}_Percentage' : percentages
    })
    summary = pd.concat([summary, temp_df], axis=1)
print(summary)

        Pclass_Count  Pclass_Percentage  Sex_Count  Sex_Percentage  \
3              491.0          55.106622        NaN             NaN   
1              216.0          24.242424        NaN             NaN   
2              184.0          20.650954        NaN             NaN   
male             NaN                NaN      577.0       64.758698   
female           NaN                NaN      314.0       35.241302   
S                NaN                NaN        NaN             NaN   
C                NaN                NaN        NaN             NaN   
Q                NaN                NaN        NaN             NaN   

        Embarked_Count  Embarked_Percentage  
3                  NaN                  NaN  
1                  NaN                  NaN  
2                  NaN                  NaN  
male               NaN                  NaN  
female             NaN                  NaN  
S                644.0            72.278339  
C                168.0            18.855219  
Q

In [31]:
female_firstclass_over30 = df[(df['Sex'] == 'female') & (df['Pclass'] == 1) & (df['Age'] > 30)]
female_firstclass_over30 = female_firstclass_over30.sort_values(by='Fare', ascending=False)
print(female_firstclass_over30.head(10))

     PassengerId  Survived  Pclass  \
258          259         1       1   
299          300         1       1   
716          717         1       1   
380          381         1       1   
779          780         1       1   
318          319         1       1   
856          857         1       1   
268          269         1       1   
609          610         1       1   
195          196         1       1   

                                                  Name     Sex   Age  SibSp  \
258                                   Ward, Miss. Anna  female  35.0      0   
299    Baxter, Mrs. James (Helene DeLaudeniere Chaput)  female  50.0      0   
716                      Endres, Miss. Caroline Louise  female  38.0      0   
380                              Bidois, Miss. Rosalie  female  42.0      0   
779  Robert, Mrs. Edward Scott (Elisabeth Walton Mc...  female  43.0      0   
318                           Wick, Miss. Mary Natalie  female  31.0      0   
856         Wick, Mrs. Georg

In [32]:
import numpy as np


# Age stats (ignoring missing)
age_mean = df['Age'].mean()
age_median = df['Age'].median()
age_mode = df['Age'].mode()[0] if not df['Age'].mode().empty else None
print(f"Age - Mean: {age_mean:.2f}, Median: {age_median:.2f}, Mode: {age_mode}")


# Mean Fare per class
fare_by_pclass = df.groupby('Pclass')['Fare'].mean()
print("Mean Fare per Pclass:")
print(fare_by_pclass)


# Survival rates
overall_survival = df['Survived'].mean()
survival_by_sex = df.groupby('Sex')['Survived'].mean()
print(f"Overall Survival Rate: {overall_survival:.2%}")
print("Survival Rate by Sex:")
print(survival_by_sex)


Age - Mean: 29.70, Median: 28.00, Mode: 24.0
Mean Fare per Pclass:
Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64
Overall Survival Rate: 38.38%
Survival Rate by Sex:
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [33]:
# 6. Missing value imputation

print("Missing Age before imputation:", df['Age'].isnull().sum())

# Compute median Age by Pclass and Sex
age_medians = df.groupby(['Pclass', 'Sex'])['Age'].median()

# Create Age_imputed by filling missing with group medians
df['Age_imputed'] = df['Age'].fillna(df.apply(lambda row: age_medians.loc[(row['Pclass'], row['Sex'])] if pd.isnull(row['Age']) else row['Age'], axis=1))

print("Missing Age_imputed after imputation:", df['Age_imputed'].isnull().sum())

Missing Age before imputation: 177
Missing Age_imputed after imputation: 0


In [36]:
# Just for understanding 
# Example of median imputation based on groups

import pandas as pd
import numpy as np

# Sample data mimicking Age, Pclass, and Sex columns with some missing Age values
data = {
    'Pclass': [1, 1, 2, 2, 3, 3, 3],
    'Sex': ['male', 'female', 'male', 'female', 'male', 'female', 'female'],
    'Age': [22, np.nan, 30, 28, np.nan, 18, np.nan]
}

df_sample = pd.DataFrame(data)

print("Original data:")
print(df_sample)

# Compute median Age grouped by Pclass and Sex
age_medians = df_sample.groupby(['Pclass', 'Sex'])['Age'].median()

print("\nMedian ages by group:")
print(age_medians)

# Impute missing Age with group median
df_sample['Age_imputed'] = df_sample.apply(
    lambda row: age_medians.loc[(row['Pclass'], row['Sex'])] if pd.isnull(row['Age']) else row['Age'], axis=1)

print("\nData after median imputation:")
print(df_sample)


Original data:
   Pclass     Sex   Age
0       1    male  22.0
1       1  female   NaN
2       2    male  30.0
3       2  female  28.0
4       3    male   NaN
5       3  female  18.0
6       3  female   NaN

Median ages by group:
Pclass  Sex   
1       female     NaN
        male      22.0
2       female    28.0
        male      30.0
3       female    18.0
        male       NaN
Name: Age, dtype: float64

Data after median imputation:
   Pclass     Sex   Age  Age_imputed
0       1    male  22.0         22.0
1       1  female   NaN          NaN
2       2    male  30.0         30.0
3       2  female  28.0         28.0
4       3    male   NaN          NaN
5       3  female  18.0         18.0
6       3  female   NaN         18.0


In [37]:
# 7. Feature extraction from text
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
print("Title counts:")
print(df['Title'].value_counts())

Title counts:
Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Ms            1
Mme           1
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64


In [38]:

# 8. Family size & new feature
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = df['FamilySize'] == 1
survival_by_isalone = df.groupby('IsAlone')['Survived'].mean()
print("Survival rate by IsAlone:")
print(survival_by_isalone)

Survival rate by IsAlone:
IsAlone
False    0.505650
True     0.303538
Name: Survived, dtype: float64


In [40]:

# 9. Pivot table and multi-index groupby
# Pivot table: Survival rate by Pclass (rows) and Sex (columns)
pivot_survival = df.pivot_table(values='Survived', index='Pclass', columns='Sex', aggfunc='mean')
print("Pivot table - Survival rate by Pclass and Sex:")
print(pivot_survival)

# Groupby: Average Fare and Age_imputed by Pclass and Embarked
groupby_avg = df.groupby(['Pclass', 'Embarked'])[['Fare', 'Age_imputed']].mean()
print("\nGroupby - Average Fare and Age_imputed by Pclass and Embarked:")
print(groupby_avg)

Pivot table - Survival rate by Pclass and Sex:
Sex       female      male
Pclass                    
1       0.968085  0.368852
2       0.921053  0.157407
3       0.500000  0.135447

Groupby - Average Fare and Age_imputed by Pclass and Embarked:
                       Fare  Age_imputed
Pclass Embarked                         
1      C         104.718529    37.988235
       Q          90.000000    38.500000
       S          70.364862    38.271024
2      C          25.358335    23.617647
       Q          12.350000    38.333333
       S          20.327439    30.355671
3      C          11.214083    21.983636
       Q          11.183393    24.194444
       S          14.644083    25.453258


In [41]:

# 10. String cleaning & parsing
df['Cabin'] = df['Cabin'].fillna('Unknown')
df['CabinDeck'] = df['Cabin'].str[0]
survival_by_cabindeck = df.groupby('CabinDeck')['Survived'].mean()
print("Survival rate by CabinDeck:")
print(survival_by_cabindeck)


Survival rate by CabinDeck:
CabinDeck
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
U    0.299854
Name: Survived, dtype: float64


In [42]:

# 11. Advanced joins / merges
# Create lookup DataFrame with Ticket frequency
ticket_lookup = df.groupby('Ticket').size().reset_index(name='TicketCount')

# Merge back into the main DataFrame
df = df.merge(ticket_lookup, on='Ticket', how='left')

# Show the top 10 tickets by TicketCount
print("Top 10 tickets by TicketCount:")
print(ticket_lookup.sort_values(by='TicketCount', ascending=False).head(10))

Top 10 tickets by TicketCount:
           Ticket  TicketCount
80           1601            7
568      CA. 2343            7
333        347082            7
566       CA 2144            6
249       3101295            6
337        347088            6
480        382652            5
621  S.O.C. 14879            5
395        349909            4
194          2666            4


In [43]:
# 12. Outlier detection & handling
fare_99 = df['Fare'].quantile(0.99)
df['Fare_outlier'] = df['Fare'] > fare_99
df['Fare'] = df['Fare'].where(df['Fare'] <= fare_99, fare_99)
print(f"99th percentile Fare: {fare_99}")
print("Outliers flagged and replaced.")

99th percentile Fare: 249.00622000000035
Outliers flagged and replaced.


In [44]:
# 13. Complex aggregation with apply
import pandas as pd

# Create AgeGroup column
def categorize_age(age):
    if pd.isnull(age):
        return 'Unknown'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    else:
        return 'Adult'

df['AgeGroup'] = df['Age'].apply(categorize_age)

# Show survival rate by AgeGroup and Sex
survival_by_agegroup_sex = df.groupby(['AgeGroup', 'Sex'])['Survived'].mean()
print("Survival rate by AgeGroup and Sex:")
print(survival_by_agegroup_sex)

Survival rate by AgeGroup and Sex:
AgeGroup  Sex   
Adult     female    0.774194
          male      0.184066
Child     female    0.593750
          male      0.567568
Teen      female    0.790698
          male      0.096154
Unknown   female    0.679245
          male      0.129032
Name: Survived, dtype: float64


In [45]:
# Add this as a new cell

# 14. Multi-step pipeline
# Fill missing Embarked with mode
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Impute Age using Pclass + Sex median
age_medians = df.groupby(['Pclass', 'Sex'])['Age'].median()
df['Age'] = df['Age'].fillna(df.apply(lambda row: age_medians.loc[(row['Pclass'], row['Sex'])] if pd.isnull(row['Age']) else row['Age'], axis=1))

# Encode Sex to numeric
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Drop unused columns
df.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Output the cleaned DataFrame
df_clean = df.copy()
print("df_clean shape:", df_clean.shape)
print("df_clean info:")
print(df_clean.info())

df_clean shape: (891, 17)
df_clean info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   891 non-null    int64  
 1   Survived      891 non-null    int64  
 2   Pclass        891 non-null    int64  
 3   Sex           891 non-null    int64  
 4   Age           891 non-null    float64
 5   SibSp         891 non-null    int64  
 6   Parch         891 non-null    int64  
 7   Fare          891 non-null    float64
 8   Embarked      891 non-null    object 
 9   Age_imputed   891 non-null    float64
 10  Title         891 non-null    object 
 11  FamilySize    891 non-null    int64  
 12  IsAlone       891 non-null    bool   
 13  CabinDeck     891 non-null    object 
 14  TicketCount   891 non-null    int64  
 15  Fare_outlier  891 non-null    bool   
 16  AgeGroup      891 non-null    object 
dtypes: bool(2), float64(3), int64(8)

In [46]:
# Add this as a new cell

# 15. Challenge analysis / storytelling

# ## Analysis: Which combination of features (at most 3) is most associated with survival?
# 
# I chose `Pclass`, `Sex`, and `AgeGroup` as they showed strong correlations in prior tasks (e.g., higher survival for females in 1st class, lower for adults).
# 
# ### Key Insights:
# - Females in 1st class have the highest survival rates.
# - Males in 3rd class have the lowest.
# - Children and teens show higher survival than adults, especially in higher classes.
# 
# ### Code and Results:

# Survival rate by Pclass, Sex, and AgeGroup
survival_analysis = df.groupby(['Pclass', 'Sex', 'AgeGroup'])['Survived'].mean().unstack().fillna(0)
print("Survival rate by Pclass, Sex, and AgeGroup:")
print(survival_analysis)

# Pivot table for clearer view
pivot_survival_full = df.pivot_table(values='Survived', index=['Pclass', 'AgeGroup'], columns='Sex', aggfunc='mean')
print("\nPivot table - Survival rate by Pclass, AgeGroup, and Sex:")
print(pivot_survival_full)

# ## Conclusion:
# The combination of `Pclass`, `Sex`, and `AgeGroup` is most associated with survival. Females in 1st class (especially adults) had ~97% survival, while males in 3rd class (adults) had ~13%. This suggests class and gender were key factors, with age playing a role for children.

Survival rate by Pclass, Sex, and AgeGroup:
AgeGroup       Adult     Child      Teen   Unknown
Pclass Sex                                        
1      0    0.382979  1.000000  0.250000  0.238095
       1    0.971831  0.000000  1.000000  1.000000
2      0    0.062500  1.000000  0.100000  0.222222
       1    0.896552  1.000000  1.000000  1.000000
3      0    0.136842  0.360000  0.078947  0.095745
       1    0.403509  0.478261  0.590909  0.595238

Pivot table - Survival rate by Pclass, AgeGroup, and Sex:
Sex                     0         1
Pclass AgeGroup                    
1      Adult     0.382979  0.971831
       Child     1.000000  0.000000
       Teen      0.250000  1.000000
       Unknown   0.238095  1.000000
2      Adult     0.062500  0.896552
       Child     1.000000  1.000000
       Teen      0.100000  1.000000
       Unknown   0.222222  1.000000
3      Adult     0.136842  0.403509
       Child     0.360000  0.478261
       Teen      0.078947  0.590909
       Unknown   0.09